In [1]:
!pip install fastapi uvicorn[standard] pyngrok transformers torch torchvision pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [13]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from PIL import Image
from io import BytesIO
from pyngrok import ngrok

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Load model + processor once
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
model.eval()
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # in production, use exact frontend URL
    allow_methods=["*"],
    allow_headers=["*"],
)

def get_caption(image):
    inputs = processor(image, return_tensors="pt")
    with torch.no_grad():
        out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

def load_image_from_bytes(image_bytes: bytes) -> Image.Image:
    return Image.open(BytesIO(image_bytes)).convert("RGB")

@app.post("/caption/")
async def caption_image(file: UploadFile = File(...)):
    image_bytes = await file.read()
    try:
        image = load_image_from_bytes(image_bytes)
        print(image)
        caption = get_caption(image)
        return {"caption": caption}
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=400)

In [10]:
!ngrok config add-authtoken 2zHLQknySY9ApId9xO4btcsD2LE_731SmjsEv86QeG4Fvsn93

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
from pyngrok import ngrok

# Make sure any previous tunnel is closed
ngrok.kill()

public_url = ngrok.connect(8000)  # port FastAPI runs on
print("New public URL:", public_url)


New public URL: NgrokTunnel: "https://7cbb-35-227-97-104.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import uvicorn
import nest_asyncio
nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [752]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     110.39.161.152:0 - "POST /caption/ HTTP/1.1" 422 Unprocessable Entity


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-21' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

<PIL.Image.Image image mode=RGB size=6000x4000 at 0x7E0CAEC28390>
INFO:     110.39.161.152:0 - "POST /caption/ HTTP/1.1" 200 OK
<PIL.Image.Image image mode=RGB size=6016x4016 at 0x7E0CAF725050>
INFO:     110.39.161.152:0 - "POST /caption/ HTTP/1.1" 200 OK
<PIL.Image.Image image mode=RGB size=6000x4000 at 0x7E0CAD064AD0>
INFO:     110.39.161.152:0 - "POST /caption/ HTTP/1.1" 200 OK
<PIL.Image.Image image mode=RGB size=6000x4000 at 0x7E0CAF5FD5D0>
INFO:     110.39.161.152:0 - "POST /caption/ HTTP/1.1" 200 OK
